In [1]:
%matplotlib inline

In [2]:
import os
import numpy as np
import pandas as pd
from random import sample, seed
seed(42)
np.random.seed(42)

import matplotlib.pyplot as plt


#-------------------------- set gpu using tf ---------------------------
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.Session(config=config)
#-------------------  start importing keras module ---------------------
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D, Flatten, Dense
from keras.models import Model
from keras import regularizers
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.preprocessing.image import load_img, img_to_array
from keras.datasets import cifar10
from keras.callbacks import TensorBoard
from sklearn.cross_validation import StratifiedShuffleSplit
from keras import models
from keras import optimizers
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from scipy.misc import imresize
from skimage.io import imread
from sklearn.model_selection import train_test_split
import keras.callbacks as callbacks

#-------------------------- set gpu using tf ---------------------------
from keras.backend.tensorflow_backend import set_session

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
df = pd.read_csv('./all/fer2013/fer2013.csv', dtype={'emotion':np.int32, 'pixels':str, 'Usage':str})

In [4]:
df['pixels'] = df['pixels'].apply(lambda x: np.fromstring(x,sep=' '))

In [5]:
train = df.loc[df['Usage'] == 'Training']
validation = df.loc[df['Usage'] == 'PublicTest']
test = df.loc[df['Usage'] == 'PrivateTest']

y_train = pd.get_dummies(train['emotion'])
y_train.columns = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

y_val = pd.get_dummies(validation['emotion'])
y_val.columns = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

y_test = pd.get_dummies(test['emotion'])
y_test.columns = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

In [6]:
x_train = np.vstack(train['pixels'].values)
x_validation = np.vstack(validation['pixels'].values)
x_test = np.vstack(test['pixels'].values)

In [7]:
#X_train = np.stack((np.reshape(x_train,(-1, 48, 48, 1)),)*3, axis=-2).squeeze()
X_val = np.stack((np.reshape(x_validation,(-1, 48, 48, 1)),)*3, axis=-2).squeeze()
X_test =  np.stack((np.reshape(x_test,(-1, 48, 48, 1)),)*3, axis=-2).squeeze()

In [8]:
def get_color(color_files, y, folder):
    numbers = []
    for item in color_files:
        for subitem in item.split('_'):
            if(subitem.isdigit()):
                numbers.append(subitem)

    X_color = np.empty((len(numbers), 48, 48, 3)).astype('uint8')
    y_color = np.empty((len(numbers), 7))

    for i in range(len(numbers)):

        path = folder + color_files[i]
        val = int(numbers[i])

        X_color[i] = np.array(imread(path)).astype('uint8')
        y_color[i] = y.iloc[val]
        
    return X_color, y_color, numbers

    #np.save('train_data', colorVal)

In [9]:
train_color_files = [k for k in os.listdir('train_color/') if '_color' in k]

X_train, y_train, indices_train = get_color(train_color_files, y_train, './train_color/')

In [10]:
X_train, X_testval, y_train, y_testval = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

In [11]:
X_val, X_test, y_val, y_test = train_test_split(X_testval, y_testval, test_size=0.5, random_state=0)

In [12]:
with tf.device('/cpu:0'):
    tf_x_train = tf.placeholder(tf.float32, shape=(len(X_train), 48, 48, 3))
    tf_x_validation = tf.placeholder(tf.float32, shape=(len(X_val), 48, 48, 3))
    tf_x_test = tf.placeholder(tf.float32, shape=(len(X_test), 48, 48, 3))

    tf_x_train_resized = tf.image.resize_images(tf_x_train,  size=[71,71])
    tf_x_validation_resized = tf.image.resize_images(tf_x_validation,  size=[71,71])
    tf_x_test_resized = tf.image.resize_images(tf_x_test,  size=[71,71])

In [13]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    X_train,X_val,X_test = sess.run([tf_x_train_resized,tf_x_validation_resized,tf_x_test_resized], feed_dict={tf_x_train: X_train,
                                                   tf_x_validation: X_val,
                                                   tf_x_test: X_test
                                                  })

In [14]:
np.shape(X_train)

(5091, 71, 71, 3)

In [15]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [16]:
#datagen = ImageDataGenerator(
#    samplewise_center=True,
#    samplewise_std_normalization=True,
#    horizontal_flip=True,
#    vertical_flip=False)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train)

In [17]:
model_imagenet = applications.xception.Xception(include_top=False, weights='imagenet', input_shape=(71, 71, 3))

In [18]:
model_imagenet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 71, 71, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 35, 35, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 35, 35, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 35, 35, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [19]:
model_imagenet.layers[-1].name

'block14_sepconv2_act'

In [20]:
#Add new classification layers
x = model_imagenet.layers[-1].output
x = Flatten()(x)
x = Dense(7)(x)
x = Activation('softmax', name='new_loss')(x)

#new Model
model = Model(model_imagenet.inputs, x, name='model_new')

In [21]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 71, 71, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 35, 35, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 35, 35, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 35, 35, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [22]:
#freeze layers
#for layer in model.layers[:-1]:
#    layer.trainable = False

for layer in model.layers:
    print(layer.name, layer.trainable)

input_1 False
block1_conv1 True
block1_conv1_bn True
block1_conv1_act True
block1_conv2 True
block1_conv2_bn True
block1_conv2_act True
block2_sepconv1 True
block2_sepconv1_bn True
block2_sepconv2_act True
block2_sepconv2 True
block2_sepconv2_bn True
conv2d_1 True
block2_pool True
batch_normalization_1 True
add_1 True
block3_sepconv1_act True
block3_sepconv1 True
block3_sepconv1_bn True
block3_sepconv2_act True
block3_sepconv2 True
block3_sepconv2_bn True
conv2d_2 True
block3_pool True
batch_normalization_2 True
add_2 True
block4_sepconv1_act True
block4_sepconv1 True
block4_sepconv1_bn True
block4_sepconv2_act True
block4_sepconv2 True
block4_sepconv2_bn True
conv2d_3 True
block4_pool True
batch_normalization_3 True
add_3 True
block5_sepconv1_act True
block5_sepconv1 True
block5_sepconv1_bn True
block5_sepconv2_act True
block5_sepconv2 True
block5_sepconv2_bn True
block5_sepconv3_act True
block5_sepconv3 True
block5_sepconv3_bn True
add_4 True
block6_sepconv1_act True
block6_sepconv1 

In [23]:
print(model_imagenet.output.shape)
print(model.output.shape)

(?, 3, 3, 2048)
(?, 7)


Now, we compile our model and train it:

In [24]:
batch_size_val = 2**7

In [25]:
batch_size_val

128

In [26]:
tbCallBack = callbacks.TensorBoard(log_dir = "./tensorboard/")
early_stopping_callback = callbacks.EarlyStopping(monitor='val_acc', patience=5, mode='max')
checkpoint_callback = callbacks.ModelCheckpoint('xception_fine_tunning.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [27]:
# Compile model and train it.

model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr = 0.0001), metrics=["accuracy"])

history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size_val),
                              validation_data=datagen.flow(X_val, y_val, batch_size=batch_size_val), 
                              steps_per_epoch=len(X_train) / batch_size_val, epochs=100, callbacks=[early_stopping_callback, checkpoint_callback, tbCallBack])

Epoch 1/100
40/39 [==============================] - 43s 1s/step - loss: 1.8070 - acc: 0.2627 - val_loss: 1.6719 - val_acc: 0.3538

Epoch 00001: val_acc improved from -inf to 0.35380, saving model to xception_fine_tunning.h5
Epoch 2/100
40/39 [==============================] - 27s 676ms/step - loss: 1.6206 - acc: 0.3562 - val_loss: 1.5483 - val_acc: 0.3996

Epoch 00002: val_acc improved from 0.35380 to 0.39963, saving model to xception_fine_tunning.h5
Epoch 3/100
40/39 [==============================] - 27s 678ms/step - loss: 1.5132 - acc: 0.4049 - val_loss: 1.5113 - val_acc: 0.4180

Epoch 00003: val_acc improved from 0.39963 to 0.41797, saving model to xception_fine_tunning.h5
Epoch 4/100
40/39 [==============================] - 27s 668ms/step - loss: 1.4273 - acc: 0.4460 - val_loss: 1.4702 - val_acc: 0.4436

Epoch 00004: val_acc improved from 0.41797 to 0.44363, saving model to xception_fine_tunning.h5
Epoch 5/100
40/39 [==============================] - 27s 668ms/step - loss: 1.3469

Finally, let's evaluate on our test set:

In [28]:
# Evaluate on validation:
# ...
print(model.metrics_names)
print(model.evaluate_generator(datagen.flow(X_val, y_val, batch_size=batch_size_val), steps=len(X_val)/batch_size_val))

['loss', 'acc']
[1.8963051810382594, 0.5160403297539702]


In [29]:
# Evaluate on test:
# ...
print(model.metrics_names)
print(model.evaluate_generator(datagen.flow(X_test, y_test, batch_size=batch_size_val), steps=len(X_test)/batch_size_val))

['loss', 'acc']
[1.999947916020404, 0.49908424832008697]
